<a href="https://colab.research.google.com/github/ybeth-gonzaga/Taylor-Swift-Spotify-Analysis/blob/main/01_initial_table_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import seaborn as sns
import pandas as pd
import numpy as np

# Initial Check for raw data

In [ ]:
# Import File
charts = pd.read_csv("/content/ph_spotify_daily_charts.csv")
charts.head()
# Original file here: https://drive.google.com/file/d/1p1fHMKZQxRxngVvRFw8iuoPP7CdElGn1/view?usp=sharing

,date,position,track_id,track_name,artist,streams
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656


In [ ]:
charts.shape
# There are 495,798 rows and 6 columns

(495798, 6)

In [ ]:
# Show the datatypes of each column
charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495798 entries, 0 to 495797
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        495798 non-null  object
 1   position    495798 non-null  int64 
 2   track_id    495798 non-null  object
 3   track_name  495788 non-null  object
 4   artist      495788 non-null  object
 5   streams     495798 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 22.7+ MB


In [ ]:
# Getting the date range
smallest_date = charts['date'].min()
largest_date = charts['date'].max()
print('The data ranges from',smallest_date,'to',largest_date)

The data ranges from 2017-01-01 to 2023-10-15


In [ ]:
# Get the subset
charts_ts = charts[charts.artist == "Taylor Swift"]
charts_ts.head()

,date,position,track_id,track_name,artist,streams
31292,2017-06-09,93,0ug5NqcwcFR2xrfTkc7k8e,Style,Taylor Swift,30552
31294,2017-06-09,95,1u8c2t2Cy7UBoG4ArRcF5g,Blank Space,Taylor Swift,29818
31341,2017-06-09,142,0cqRj7pUJDkTCEsJkx8snD,Shake It Off,Taylor Swift,24406
31346,2017-06-09,147,1vrd6UOGamcKNGnSHJQlSt,Love Story,Taylor Swift,23608
31348,2017-06-09,149,3fVnlF4pGqWI9flVENcT28,Wildest Dreams,Taylor Swift,23323


In [ ]:
# Download the subset file
charts_ts.to_csv("ph_spotify_daily_charts_ts.csv")

# Initial Check for Subset Data (Taylor Swift songs)

In [ ]:
charts_ts.shape
# There are 19,572 rows and 6 columns

(19572, 6)

In [ ]:
# Getting the date range
smallest_date = charts_ts['date'].min()
largest_date = charts_ts['date'].max()
print('The data ranges from',smallest_date,'to',largest_date)

The data ranges from 2017-06-09 to 2023-10-15


In [ ]:
# Show the datatypes of each column
charts_ts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19572 entries, 31292 to 495775
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        19572 non-null  object
 1   position    19572 non-null  int64 
 2   track_id    19572 non-null  object
 3   track_name  19572 non-null  object
 4   artist      19572 non-null  object
 5   streams     19572 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.5+ MB


In [ ]:
# Identify how many unique dates are to be analyzed
charts_ts['date'].nunique()

1976

In [ ]:
charts_ts['date'].unique()

array(['2017-06-09', '2017-06-10', '2017-06-11', ..., '2023-10-13',
       '2023-10-14', '2023-10-15'], dtype=object)

# Create daily summary for T.Swift data

In [ ]:
# Create a dataframe containing the unique dates
charts_ts_summary = pd.DataFrame({'date': charts['date'].unique()})
charts_ts_summary.head()

,date
0,2017-01-01
1,2017-01-02
2,2017-01-03
3,2017-01-04
4,2017-01-05


In [ ]:
# Get the total streams per date
total_streams_per_date = []

for date in charts_ts_summary['date']:
    total_streams = charts_ts[charts_ts['date'] == date]['streams'].sum()
    total_streams_per_date.append(total_streams)

charts_ts_summary['total_streams'] = total_streams_per_date
charts_ts_summary.head()

,date,total_streams
0,2017-01-01,0
1,2017-01-02,0
2,2017-01-03,0
3,2017-01-04,0
4,2017-01-05,0


In [ ]:
# Get the percent change of the daily streams
charts_ts_summary['stream_percent_change'] = charts_ts_summary['total_streams'].pct_change()*100
charts_ts_summary['stream_percent_change'].fillna(0, inplace=True)

In [ ]:
charts_ts_summary.tail()

,date,total_streams,stream_percent_change
2474,2023-10-11,2623054,-1.468291
2475,2023-10-12,2758485,5.163104
2476,2023-10-13,2769349,0.393839
2477,2023-10-14,3277303,18.342000
2478,2023-10-15,3463565,5.683393


In [ ]:
# Get the total number of rows per date
total_songs_per_date = []

for date in charts_ts_summary['date']:
    # get the number of rows with that date
    total_songs = charts_ts[charts_ts['date'] == date].shape[0]
    total_songs_per_date.append(total_songs)

charts_ts_summary['total_count'] = total_songs_per_date

In [ ]:
charts_ts_summary.tail()

,date,total_streams,stream_percent_change,total_count
2474,2023-10-11,2623054,-1.468291,20
2475,2023-10-12,2758485,5.163104,21
2476,2023-10-13,2769349,0.393839,21
2477,2023-10-14,3277303,18.342000,23
2478,2023-10-15,3463565,5.683393,24


In [ ]:
# Get the percent change of the daily count of songs in the top 200
charts_ts_summary['count_percent_change'] = charts_ts_summary['total_count'].pct_change()*100
charts_ts_summary['count_percent_change'].fillna(0, inplace=True)

In [ ]:
# Get the total streams per date
highest_rank_per_date = []

for date in charts_ts_summary['date']:
    highest_rank_daily = charts_ts[charts_ts['date'] == date]['position'].min()
    highest_rank_per_date.append(highest_rank_daily)

charts_ts_summary['highest_rank'] = highest_rank_per_date

In [ ]:
charts_ts_summary.tail()

,date,total_streams,stream_percent_change,total_count,count_percent_change,highest_rank
2474,2023-10-11,2623054,-1.468291,20,-4.761905,5.0
2475,2023-10-12,2758485,5.163104,21,5.000000,4.0
2476,2023-10-13,2769349,0.393839,21,0.000000,3.0
2477,2023-10-14,3277303,18.342000,23,9.523810,3.0
2478,2023-10-15,3463565,5.683393,24,4.347826,3.0


In [ ]:
# Get the percent change of the daily count of songs in the top 200
charts_ts_summary['top_rank_diff'] = charts_ts_summary['highest_rank'].diff()
charts_ts_summary['top_rank_diff'].fillna(0, inplace=True)

In [ ]:
charts_ts_summary.tail()

,date,total_streams,stream_percent_change,total_count,count_percent_change,highest_rank,top_rank_diff
2474,2023-10-11,2623054,-1.468291,20,-4.761905,5.0,-1.0
2475,2023-10-12,2758485,5.163104,21,5.000000,4.0,-1.0
2476,2023-10-13,2769349,0.393839,21,0.000000,3.0,-1.0
2477,2023-10-14,3277303,18.342000,23,9.523810,3.0,0.0
2478,2023-10-15,3463565,5.683393,24,4.347826,3.0,0.0


In [ ]:
# Download the subset file
charts_ts_summary.to_csv("ph_spotify_daily_charts_ts_summary.csv")